# Coursework 1

This notebook is intended to be used as a starting point for your experiments. The instructions can be found in the MLP2024_25_CW1_Spec.pdf (see Learn,  Assignment Submission, Coursework 1). The methods provided here are just helper functions. If you want more complex graphs such as side by side comparisons of different experiments you should learn more about matplotlib and implement them. Before each experiment remember to re-initialize neural network weights and reset the data providers so you get a properly initialized experiment. For each experiment try to keep most hyperparameters the same except the one under investigation so you can understand what the effects of each are.

In [45]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

def train_model_and_plot_stats(
        model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True):
    
    # As well as monitoring the error over training also monitor classification
    # accuracy i.e. proportion of most-probable predicted classes being equal to targets
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

    # Use the created objects to initialise a new Optimiser instance.
    optimiser = Optimiser(
        model, error, learning_rule, train_data, valid_data, data_monitors, notebook=notebook)

    # Run the optimiser for num_epochs epochs (full passes through the training set)
    # printing statistics every epoch.
    stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)

    # # Plot the change in the validation and training set error over training.
    # fig_1 = plt.figure(figsize=(8, 4))
    # ax_1 = fig_1.add_subplot(111)
    # for k in ['error(train)', 'error(valid)']:
    #     ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
    #               stats[1:, keys[k]], label=k)
    # ax_1.legend(loc=0)
    # ax_1.set_xlabel('Epoch number')
    # ax_1.set_ylabel('Error')

    # # Plot the change in the validation and training set accuracy over training.
    # fig_2 = plt.figure(figsize=(8, 4))
    # ax_2 = fig_2.add_subplot(111)
    # for k in ['acc(train)', 'acc(valid)']:
    #     ax_2.plot(np.arange(1, stats.shape[0]) * stats_interval, 
    #               stats[1:, keys[k]], label=k)
    # ax_2.legend(loc=0)
    # ax_2.set_xlabel('Epoch number')
    # ax_2.set_ylabel('Accuracy')
    
    # return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2

    return stats, keys, run_time

In [ ]:
# The below code will set up the data providers, random number
# generator and logger objects needed for training runs. As
# loading the data from file take a little while you generally
# will probably not want to reload the data providers on
# every training run. If you wish to reset their state you
# should instead use the .reset() method of the data providers.
import numpy as np
import logging
import sys
# sys.path.append('/path/to/mlpractical')
from mlp.data_providers import MNISTDataProvider, EMNISTDataProvider

# Seed a random number generator
seed = 11102019 
rng = np.random.RandomState(seed)
batch_size = 100
# Set up a logger object to print info about the training run to stdout
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = [logging.StreamHandler()]

# Create data provider objects for the MNIST data set
train_data = EMNISTDataProvider('train', batch_size=batch_size, rng=rng)
valid_data = EMNISTDataProvider('valid', batch_size=batch_size, rng=rng)

In [ ]:
# The model set up code below is provided as a starting point.
# You will probably want to add further code cells for the
# different experiments you run.

# %pip install tqdm

from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit
from mlp.learning_rules import AdamLearningRule
from mlp.optimisers import Optimiser

def Experiment_Width(widths=[32, 64, 128]):

    stats_list = []

    # Setup hyperparameters
    learning_rate = 0.001
    num_epochs = 100
    stats_interval = 1

    for width in widths:

        input_dim, output_dim, hidden_dim = 784, 47, width

        weights_init = GlorotUniformInit(rng=rng)
        biases_init = ConstantInit(0.)

        # Create model with ONE hidden layer
        model = MultipleLayerModel([
            AffineLayer(input_dim, hidden_dim, weights_init, biases_init), # hidden layer
            ReluLayer(),
            AffineLayer(hidden_dim, output_dim, weights_init, biases_init) # output layer
        ])

        error = CrossEntropySoftmaxError()
        # Use a Adam learning rule
        learning_rule = AdamLearningRule(learning_rate=learning_rate)

        print(f"------------------------- Total Epoch: {num_epochs}, Width: {width} -------------------------")

        # Remember to use notebook=False when you write a script to be run in a terminal
        stats, keys, run_time = train_model_and_plot_stats(
            model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)
        
        stats_list.append(stats)

    # Plot the change in the validation and training set error over training.
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    
    for k in ['error(train)', 'error(valid)']:

        if k == 'error(train)':
                linestyle = "-"
                label = "(train)"
        else: 
            linestyle = "--"
            label = "(valid)"

        for idx, stats in enumerate(stats_list):

            print(f"Width: {widths[idx]}, Final error{label}: {stats[-1, keys[k]]}")

            ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                    stats[1:, keys[k]], label=f"width {widths[idx]}{label}", linestyle=linestyle)
            
            
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')
    ax_1.set_ylabel('Error')

    plt.savefig("../report/figures/empty_error_curve_width.png")

    # Plot the change in the validation and training set accuracy over training.
    fig_2 = plt.figure(figsize=(8, 4))
    ax_2 = fig_2.add_subplot(111)

    for k in ['acc(train)', 'acc(valid)']:

        if k == 'acc(train)':
                linestyle = "-"
                label = "(train)"
        else: 
            linestyle = "--"
            label = "(valid)"

        for idx, stats in enumerate(stats_list):

            print(f"Width: {widths[idx]}, Final error{label}: {stats[-1, keys[k]]}")

            ax_2.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                    stats[1:, keys[k]], label=f"width {widths[idx]}{label}", linestyle=linestyle)
        
    ax_2.legend(loc=0)
    ax_2.set_xlabel('Epoch number')
    ax_2.set_ylabel('Accuracy')

    plt.savefig("../report/figures/empty_acc_curve_width.png")

Experiment_Width()


In [ ]:
def Experiment_Depth(depths=[1, 2, 3]):

    stats_list = []

    # Setup hyperparameters
    learning_rate = 0.001
    num_epochs = 2
    stats_interval = 1
    input_dim, output_dim, hidden_dim = 784, 47, 128

    for depth in depths:

        weights_init = GlorotUniformInit(rng=rng)
        biases_init = ConstantInit(0.)

        model_list = [AffineLayer(input_dim, hidden_dim, weights_init, biases_init), ReluLayer()] # init
        for _ in range(depth-1):
            model_list.append(AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init))
            model_list.append(ReluLayer())
        model_list.append(AffineLayer(hidden_dim, output_dim, weights_init, biases_init))

        print(model_list)

        # Create model with TWO hidden layers
        model = MultipleLayerModel(model_list)

        error = CrossEntropySoftmaxError()
        # Use a Adam learning rule
        learning_rule = AdamLearningRule(learning_rate=learning_rate)

        print(f"------------------------- Total Epoch: {num_epochs}, Didth: {depth} -------------------------")

        # Remember to use notebook=False when you write a script to be run in a terminal
        stats, keys, run_time = train_model_and_plot_stats(
            model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True)
        
        stats_list.append(stats)

    # Plot the change in the validation and training set error over training.
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    
    for k in ['error(train)', 'error(valid)']:

        if k == 'error(train)':
                linestyle = "-"
                label = "(train)"
        else: 
            linestyle = "--"
            label = "(valid)"

        for idx, stats in enumerate(stats_list):

            print(f"Depth: {depths[idx]}, Final error{label}: {stats[-1, keys[k]]}")

            ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                    stats[1:, keys[k]], label=f"depth {depths[idx]}{label}", linestyle=linestyle)
            
            
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')
    ax_1.set_ylabel('Error')

    plt.savefig("../report/figures/empty_error_curve_depth.png")

    # Plot the change in the validation and training set accuracy over training.
    fig_2 = plt.figure(figsize=(8, 4))
    ax_2 = fig_2.add_subplot(111)

    for k in ['acc(train)', 'acc(valid)']:

        if k == 'acc(train)':
                linestyle = "-"
                label = "(train)"
        else: 
            linestyle = "--"
            label = "(valid)"

        for idx, stats in enumerate(stats_list):

            print(f"Depth: {depths[idx]}, Final acc{label}: {stats[-1, keys[k]]}")

            ax_2.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                    stats[1:, keys[k]], label=f"depth {depths[idx]}{label}", linestyle=linestyle)
        
    ax_2.legend(loc=0)
    ax_2.set_xlabel('Epoch number')
    ax_2.set_ylabel('Accuracy')

    plt.savefig("../report/figures/empty_acc_curve_depth.png")

Experiment_Depth()